# Project: Sharktale, SharkTrack
## Data Cleaning

In [ ]:
!pip install xlrd #!pip install country_converter, keep this here pls

### Import of Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')

### Loading the Excel Data

In [ ]:
#define path to xls
url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
df = pd.read_excel(url)
df

### Data at a Glance 
Reviewing how the database is organised by using .head(), .info(), .describe(), .tail(), .columns. This will help me on how to pepare the data cleaning process.

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

### Columns Cleaning and Renaming 

I have reviewed the columns and decided to do the following: 
- Remove extra " " and also replacing extra " " with "_". 
- Dropping the following columns: "Case_Number_1", "Unnamed_21". "Unnamed_22" . 
- Renaming:  'Fatal Y/N': 'Fatal', 'Sex': 'Gender',  'pdf': 'PDF',  and  'Species ': 'Species', 'href': 'Link','original order': 'Original_Order', 

In [ ]:
df = df.rename(columns= {
    'Fatal Y/N': 'Fatal',
    'Species ': 'Species',
    'pdf': 'PDF', 
    'Sex': 'Gender',
    'href formula': 'Href_formula', 
    'href': 'Link',
    'Case Number': 'Case_Number', 
    'Case Number.1': 'Case_Number_1',
    'original order': 'Original_Order', 
    'Unnamed: 21': 'Unnamed_21', 
    'Unnamed: 22': 'Unnamed_22'
})
df.columns

In [ ]:
df["Case_Number"].isin(df["Case_Number_1"]).value_counts() # There is less than 10% difference 

In [ ]:
# Dropping Columns
df = df.drop(columns=["Unnamed_21", "Unnamed_22", "PDF", "Href_formula", "Case_Number_1"])
df.columns

In [ ]:
### Cleaning and replacing all Nan's of Columns that are objects/strings
df["Country"] = df["Country"].fillna("Unknown").str.strip().astype(str)
df["State"] = df["State"].fillna("Unknown").str.strip().astype(str)
df["Location"] = df["Location"].fillna("Unknown").str.strip().astype(str)
df["Activity"] = df["Activity"].fillna("Unknown").str.strip().astype(str)
df["Name"] = df["Name"].fillna("Unknown").str.strip().astype(str)
df["Gender"] = df["Gender"].fillna("Unknown").str.strip().astype(str)
df["Injury"] = df["Injury"].fillna("Unknown").str.strip().astype(str)
df["Species"] = df["Species"].fillna("Unknown").str.strip().astype(str)
df["Fatal"] = df["Fatal"].fillna("Unknown").str.strip().astype(str)
df["Link"] = df["Link"].fillna("Unknown").str.strip().astype(str)
df["Source"] = df["Source"].fillna("Unknown").str.strip().astype(str)
df["Type"] = df["Type"].fillna("Unknown").str.strip().astype(str)
df["Name"] = df["Name"].fillna("Unknown").str.strip().astype(str)
#df["Age"] = df["Age"].fillna("Unknown").str.strip().astype(str)

In [ ]:
print(df.isna().sum())  # Check-up for all Nan's

### Cleaning Column 'Month'

In [ ]:
def finding_month(date): 
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Oct", "Nov", "Dec"]
    #If it founds the info of a given month, it returns an output of this cleaning function
    try:
        for month in months: 
            if re.search(month, date):
                return month
    except TypeError:
        pass 
  
df["Month"] = df["Date"]
df["Month"] = df["Date"].apply(finding_month)
df["Month"].fillna(value="Unknown", inplace=True)
print(df["Month"].value_counts())  # Check unique values

### Cleaning Column 'Date'

- Reviewing the column 'Date'. I am stripping the time component, because we already have a column with time.

In [ ]:
df["Date"].unique()[:100]

In [ ]:
print(df["Date"].dtype)

In [ ]:
# To ensure all values are strings before applying the function 
df["Date"] = df["Date"].astype(str).str.lower().str.strip() # Converting everything to lowercase, and removing extra spaces before and after text. 

In [ ]:
def extract_date(text):
    """
    Extracts valid date-like information from messy strings.
    Converts multiple formats into a standard date format (DD-MM-YYYY).
    """
    if not isinstance(text, str) or text.lower().strip() in ["none", "nan", "null", ""]:
        return None  # Handle missing values

    text = text.strip()  # Remove leading/trailing spaces

    # Define regex patterns to capture different date formats
    date_patterns = [
        r"\d{4}-\d{2}-\d{2}",             # "2023-05-20" (ISO format)
        r"\d{1,2}-[a-z]{3}-\d{4}",        # "18-may-2023"
        r"\d{1,2} [a-z]{3}-\d{4}",        # "09 may-2023"
        r"\d{1,2} [a-z]{3} \d{4}",        # "15 Mar 2024"
        r"\d{1,2}-[A-Za-z]{3}-\d{4}",     # "23-Jun-2023"
        r"\d{1,2} [A-Za-z]{3} \d{4}",     # "15 Mar 2024"
        r"\d{1,2} [A-Za-z]+ \d{4}",       # "15 March 2024"
        r"\d{1,2}(st|nd|rd|th)? of [A-Za-z]+, \d{4}"  # "24th of May, 2022"
    ]

    for pattern in date_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(0)  # Extract the matched date part

    return None  # Return None if no valid date is found

 # Clean the "Date" column before applying regex
df["Date"] = df["Date"].astype(str).str.strip()

# Apply extraction function
df["Cleaned_Date"] = df["Date"].apply(extract_date)

# Convert extracted dates into proper datetime format
df["Cleaned_Date"] = pd.to_datetime(df["Cleaned_Date"], errors="coerce")

# Convert to DD-MM-YYYY format (Final Step)
df["Cleaned_Date"] = df["Cleaned_Date"].dt.strftime('%d-%m-%Y')

In [ ]:
# Count missing values (NaN)
null_count = df["Cleaned_Date"].isna().sum()
print(f"Number of null values: {null_count}")

In [ ]:
# Show only rows where Cleaned_Date is NaN
missing_rows = df[df["Cleaned_Date"].isna()]
missing_rows

In [ ]:
display(df)

In [ ]:
df_count = df.groupby("Year")["Date"].count()
df_count[df_count.index > 1750].plot()

In [ ]:
#Converting Datetime to Date format 

### Data Cleaning for the Column 'Type' 

In [ ]:
df.Type.unique() 

In [ ]:
print(df["Type"].value_counts()) 

In [ ]:
print(df["Type"].value_counts(dropna=False)) # I wanted to review the Nan 

In [ ]:
#Reviewing how many were provoked and unprovoked 
print(df["Type"].unique())

In [ ]:
df["Type"] = df["Type"].str.strip()
df["Type"] = df["Type"].replace({
    " Provoked": "Provoked", 
    "Unconfirmed": "Unknown",
    "?" : "Unknown",
    "Invalid": "Unknown",
    "Under investigation": "Unknown",
    "Questionable": "Unknown", 
    "Watercraft" : "Water Vehicle", 
    "Boat" : "Water Vehicle"
})

df["Type"] = df["Type"].fillna("Unknown")
print(df["Type"].unique())

In [ ]:
#Filter for Unprovoked Shark Attacks 
filter_provoked_gen = ((df["Type"] == "Unprovoked") & (df["Year"] >= 1960) & (df["Year"]<2025))
df = df.loc[filter_provoked_gen]

### Data Cleaning for the Column 'Gender'

In [ ]:
df["Gender"] = df["Gender"].replace({
    " M": "M", 
    "M ": "M", 
    "M x 2": "M", 
    " nan": "Unknown", 
    "lli": "Unknown", 
    "N": "M", 
    ".": "Unknown"
    })
df["Gender"] = df["Gender"].fillna("Unknown")
df.Gender.unique()

### Data Cleaning for the Column 'Species'

In [ ]:
df["Species"].unique()
print(df["Species"].unique()[:600])

In [ ]:
# Cleaning the Column 'Species' a bit 
df["Species"] = df["Species"].str.extract(r'([A-Za-z\s-]+)').fillna("Unknown") # Cleaning here the unnecessary details
df["Species"] = df["Species"].replace({
    "Not stated":"Unknown"
})
print(df["Species"])

In [ ]:
df["Species_Types"] = df["Species"].copy() # creating a copy of species and naming it Species_Types

In [ ]:
df["Species_Types"] = df["Species_Types"].astype(str).str.strip()

In [ ]:
df["Species_Types"] = df["Species_Types"].fillna("Unknown").astype(str)

In [ ]:
df["Species_Types"] = df["Species_Types"].replace({
    "Not stated": "Unknown",
    "Not specified": "Unknown",
    "Invalid": "Unknown",
    "Great White Shark": "White Shark"   
}) 

In [ ]:
def clean_shark_name(species): 
    species = species.strip().lower()

    match = re.search(r"(?:\b\w+\s+)*?(great white|spinner|leopard|whitetip|sandtiger|lemon|horn|white|tiger|broadnos|bull|Bull|hammerhead|hammer|grey|Grey|Hammerhead|blacktip|Blacktip|whale|nurse|mako|reef|Reef)(?:\s+\w+)*?\s*shark(?:\s+\w+)*?", species)

    if match:
        #return match.group(0).title()
        return match.group(1).title() + " Shark"
    else: 
        return "Other/Unknown"

df["Species_Types"] = df["Species_Types"].apply(clean_shark_name)

print(df["Species_Types"].value_counts())  # Check unique values
print(df["Species_Types"].tail())  # Display first few rows

In [ ]:
df["Species_Types"].unique()

In [ ]:
#df["Location"] = df["Location"].astype(str).str.strip()
print(df["Species_Types"].value_counts())

### Data Cleaning for the Column 'Fatal'

In [ ]:
df["Fatal"].unique()

In [ ]:
df["Fatal"] = df["Fatal"].fillna("Unknown").str.strip().astype(str)

In [ ]:
# Data Cleaning for The Column Fatal
df["Fatal"] = df["Fatal"].fillna("Unknown")
df["Fatal"] = df["Fatal"].str.strip()
df["Fatal"].value_counts(dropna=False)
df["Fatal"] = df["Fatal"].astype(str)
df["Fatal"] = df["Fatal"].replace({
    "Nan": "Unknown",
    " N": "NO",
    "UNKNOWN": "Unknown",
    "F": "Unknown",\
    "M": "NO",
    "n": "NO",
    "N": "NO",
    "Nq": "NO",
    "Y": "YES",
    2017: "Unknown",
    "2017 ": "Unknown",
     "Y x 2": "Yes",
     "y": "Yes",
     "N   ": "NO",
     "nan": "Unknown"
})
df["Fatal"].unique()

In [ ]:
# Define colors and labels dynamically to match data
slices = df["Fatal"].value_counts().nlargest(2)
labels = slices.index.tolist()  # Get correct labels
colors = ["skyblue", "coral"][:len(labels)]  # Ensure colors match number of slices
explode = [0] * len(labels)  # No explode by default

# Create pie chart
plt.pie(slices, labels=labels, colors=colors, explode=explode, 
        autopct="%1.1f%%", wedgeprops={"edgecolor": "black"})

plt.title("Fatality Rate of Shark Attacks (1967-2025)")
plt.tight_layout()
plt.savefig("AttacksFatalityRate.png")

# Show the chart
plt.show()


In [ ]:
#print(df["Species_Types"].value_counts())

### Data Cleaning for the Column 'Country'

In [ ]:
df["Country"].unique()

In [ ]:
len(df["Country"].unique())

In [ ]:
update_country = {
    'AUSTRALIA': 'Australia',
    'BAHAMAS': 'Bahamas',
    'BELIZE': 'Belize',
    'COLOMBIA': 'Colombia',
    'COLUMBIA': 'Colombia',
    'EGYPT': 'Egypt',
    'ENGLAND': 'United Kingdom',
    'FRENCH POLYNESIA': 'French Polynesia',
    'INDIA': 'India',
    'INDONESIA': 'Indonesia',
    'JAMAICA': 'Jamaica',
    'JAPAN': 'Japan',
    'MALDIVE ISLANDS': 'Maldives',
    'MALDIVES': 'Maldives',
    'MEXICO': 'Mexico',
    'MeXICO': 'Mexico',
    'MEXICO ': 'Mexico',
    'NEW CALEDONIA': 'New Caledonia',
    'NEW ZEALAND': 'New Zealand',
    'PHILIPPINES': 'Philippines',
    'SPAIN': 'Spain',
    'SOUTH AFRICA': 'South Africa',
    'South Africa': 'South Africa',
    'THAILAND': 'Thailand',
    'TRINIDAD': 'Trinidad and Tobago',
    'TRINIDAD & TOBAGO': 'Trinidad and Tobago',
    'TURKS & CAICOS': 'Turks and Caicos',
    'TURKS and CaICOS': 'Turks and Caicos',
    'UNITED ARAB EMIRATES (UAE)': 'United Arab Emirates',
    'UNITED KINGDOM': 'United Kingdom',
    'USA': 'United States',
    'UAE': 'United Arab Emirates',
    'UNITED STATES': 'United States',
    'REUNION ISLAND': 'Reunion',
    'REUNION': 'Reunion',
    'ST MARTIN': 'Saint Martin',
    'ST. MARTIN': 'Saint Martin',
    'ST. MAARTIN': 'Saint Martin',
    'SAINT MAARTEN': 'Saint Martin',
    'PAPUA NEW GUINEA': 'Papua New Guinea',
    'FIJI': 'Fiji',
    'Fiji': 'Fiji',
    'CANADA': 'Canada',
    'CUBA': 'Cuba',
    'ARGENTINA': 'Argentina',
    'BRAZIL': 'Brazil',
    'CHILE': 'Chile',
    'ECUADOR': 'Ecuador',
    'PERU': 'Peru',
    'VENEZUELA': 'Venezuela',
    'COOK ISLANDS': 'Cook Islands',
    'DOMINICAN REPUBLIC': 'Dominican Republic',
    'SEYCHELLES': 'Seychelles',
    'ST KITTS / NEVIS': 'Saint Kitts and Nevis',
    'ST HELENA, British overseas territory': 'Saint Helena',
    'SOLOMON ISLANDS': 'Solomon Islands',
    'TONGA': 'Tonga',
    ' TONGA': 'Tonga',
    'KIRIBATI': 'Kiribati',
    'PALAU': 'Palau',
    'MALTA': 'Malta',
    'SCOTLAND': 'United Kingdom',
    'IRELAND': 'Ireland',
    'ITALY': 'Italy',
    'MALAYSIA': 'Malaysia',
    'LIBYA': 'Libya',
    'JORDAN': 'Jordan',
    'ISRAEL': 'Israel',
    'CHINA': 'China',
    'TAIWAN': 'Taiwan',
    'INDIAN OCEAN': 'Indian Ocean',
    'INDIAN OCEAN?': 'Indian Ocean',
    'INDIAN OCEAN?': 'Indian Ocean',
    'NORTH ATLANTIC OCEAN': 'North Atlantic Ocean',
    'NORTH ATLANTIC OCEAN ': 'North Atlantic Ocean',
    'SOUTH ATLANTIC OCEAN': 'South Atlantic Ocean',
    'ATLANTIC OCEAN': 'Atlantic Ocean',
    'PACIFIC OCEAN': 'Pacific Ocean',
    'PACIFIC OCEAN ': 'Pacific Ocean',
    'SOUTH PACIFIC OCEAN': 'South Pacific Ocean',
    'NORTH PACIFIC OCEAN': 'North Pacific Ocean',
    'CARIBBEAN SEA': 'Caribbean Sea',
    'MID ATLANTIC OCEAN': 'Mid Atlantic Ocean',
    'SOUTH CHINA SEA': 'South China Sea',
    'BAY OF BENGAL': 'Bay of Bengal',
    'RED SEA': 'Red Sea',
    'RED SEA?': 'Red Sea',
    'RED SEA / INDIAN OCEAN': 'Red Sea',
    'GULF OF ADEN': 'Gulf of Aden',
    'PERSIAN GULF': 'Persian Gulf',
    'CEYLON': 'Sri Lanka',
    'CEYLON (SRI LANKA)': 'Sri Lanka',
    'SRI LANKA': 'Sri Lanka',
    'BANGLADESH': 'Bangladesh',
    'BURMA': 'Myanmar',
    'MYANMAR': 'Myanmar',
    'VIETNAM': 'Vietnam',
    'HONG KONG': 'Hong Kong',
    'MARTINIQUE': 'Martinique',
    'NETHERLANDS ANTILLES': 'Netherlands Antilles',
    'NORTHERN MARIANA ISLANDS': 'Northern Mariana Islands',
    'FEDERATED STATES OF MICRONESIA': 'Micronesia',
    'MICRONESIA': 'Micronesia',
    'FALKLAND ISLANDS': 'Falkland Islands',
    'GIBRALTAR': 'Gibraltar',
    'SAUDI ARABIA': 'Saudi Arabia',
    'SINGAPORE': 'Singapore',
    'SENEGAL': 'Senegal',
    'SOMALIA': 'Somalia',
    'SOUTH KOREA': 'South Korea',
    'NORTH KOREA': 'North Korea',
    'RUSSIA': 'Russia',
    'GREECE': 'Greece',
    'TUNISIA': 'Tunisia',
    'TURKEY': 'Turkey',
    'IRAN': 'Iran',
    'IRAQ': 'Iraq',
    'KUWAIT': 'Kuwait',
    'LEBANON': 'Lebanon',
    'SYRIA': 'Syria',
    'AFRICA': 'Africa',
    'Coast of AFRICA': 'Africa',
    'GEORGIA': 'Georgia',
    'GHANA': 'Ghana',
    'GUINEA': 'Guinea',
    'NAMIBIA': 'Namibia',
    'TANZANIA': 'Tanzania',
    'ALGERIA': 'Algeria',
    'DJIBOUTI': 'Djibouti',
    'EQUATORIAL GUINEA / CAMEROON': 'Equatorial Guinea',
    'WEST INDIES': 'Caribbean',
    'BRITISH ISLES': 'United Kingdom',
    'BRITISH WEST INDIES': 'Caribbean',
    'ST HELENA, British overseas territory': 'Saint Helena',
    'NORWAY': 'Norway',
    'ICELAND': 'Iceland',
    'GABON': 'Gabon',
    'MAYOTTE': 'Mayotte',
    'SWEDEN': 'Sweden',
    'SLOVENIA': 'Slovenia',
    'CURACAO': 'Curaçao',
    'HAITI': 'Haiti',
    'GUATEMALA': 'Guatemala',
    'NICARAGUA': 'Nicaragua',
    'NICARAGUA ': 'Nicaragua',
    'HONDURAS': 'Honduras',
    'EL SALVADOR': 'El Salvador',
    'COSTA RICA': 'Costa Rica',
    'PANAMA': 'Panama',
    'BARBADOS': 'Barbados',
    'ARUBA': 'Aruba',
    'GRAND CAYMAN': 'Cayman Islands',
    'CAYMAN ISLANDS': 'Cayman Islands',
    'SAINT LUCIA': 'Saint Lucia',
    'USA': 'United States', 
    'Usa': 'United States'
}

In [ ]:
df['Country'] = df['Country'].replace(update_country).fillna('Unknown')

In [ ]:
df['Country'].unique()

In [ ]:
 #Data Cleaning for the column 'Country', I've tried to use regex 
df["Country"] = df["Country"].str.strip().str.title()
df["Country"] = df["Country"].str.replace(r"[^a-zA-Z\s]", "", regex=True)
#df["Country"] = df["Country"].map(lambda x: x.upper())
df["Country"] = df["Country"].fillna("Unknown")
print(df["Country"].unique())

In [ ]:
unique_country = len(df['Country'].unique())
unique_country

In [ ]:
df["Country"] = df["Country"].replace(update_country) # Reassigning it back to "Country"

In [ ]:
# Top 10 Country
top_10_attacks_country = df["Country"].value_counts().nlargest(9)

# Graphic Attacks by State 
top_10_attacks_country.sort_values(ascending=True, inplace=True)
top_10_attacks_country.plot.barh(color="skyblue")

plt.title("Shark Attacks in Countries (1967 -2017)")
plt.xlabel("Attacks")
plt.grid(axis="x")
plt.tight_layout()
plt.savefig("AttacksByCountry.png") # Saving as an image 

### Data Cleaning for the Column 'States'

In [ ]:
# A filter specifically for USA 
filter_usa = (df["Country"] == "United States")
usa_df = df.loc[filter_usa]

In [ ]:
# Top Shark Attacks by State
top_attacks_usa = df["State"].value_counts()
top_attacks_usa

In [ ]:
# Top 10 Shark Attacks by State
top_10_attacks_usa = df["State"].value_counts().nlargest(9)

# Graphic Attacks by State 
top_10_attacks_usa.sort_values(ascending=True, inplace=True)
top_10_attacks_usa.plot.barh(color="skyblue")

plt.title("Shark Attacks in the U.S by State (1967 -2017)")
plt.xlabel("Attacks")
plt.grid(axis="x")
plt.tight_layout()
plt.savefig("AttacksByState.png") # Saving as an image 

### Data Cleaning for the Column 'Age'

In [ ]:
df["Age"].unique()

In [ ]:
df["Age"] = df["Age"].str.strip()
print(df["Age"].value_counts())

In [ ]:
df["Age"] = df["Age"].replace({"20s":"25", "30s":"35", "teen": "15", "Teen": "15", "mid-30s":"35", "21 or 26":"24", "60's":"65", "60s":"65", "12 or 13":"12", "50s":"55", "Middle age": "40", "9 & 12":"9", "Elderly": "12", "6½":"6"})

In [ ]:
print(df["Age"].unique()[:120])

In [ ]:
def age_process(age):
    try:
        age = int(age)
    except:
        age = 0
    if (age > 0 and age <= 100):
        return age
    else:
        return np.nan
df['Age'].fillna(0,inplace=True)
df['Age'] = df['Age'].apply(age_process)
fig,ax = plt.subplots(figsize=(8,6))
ax = sns.distplot(df['Age'].dropna().astype(np.int32),
             ax=ax,
             hist_kws={"alpha": 0.6, "color": "skyblue"},
             kde=False,bins=15)
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Age Distribution')

In [ ]:
#Graphic Attacks By Age

usa_df["Age"] = pd.to_numeric(usa_df["Age"], errors="coerce")

age_usa = usa_df["Age"].dropna()

# Making bins for age groups
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 100]
plt.hist(age_usa, bins=bins, edgecolor = "black", color="skyblue")

median = age_usa.median()
plt.axvline(median, color="coral", label= f"Median Age: {median:.1f}", linewidth=2)

# Labels and Title 
plt.title("Ages of Shark Attack VIctims in the USA (1960-2025)")
plt.ylabel("Attacks")
plt.xlabel("Age")

# Formatting
plt.legend()
plt.grid(axis="y")
plt.tight_layout()
plt.savefig("AttacksByAge.png")

### Data Cleaning for the Column 'Location'

In [ ]:
df["Location"] = df["Location"].astype(str).str.strip()
df["Location"] = df["Location"].map(lambda x:x.upper())
df["Location"] = df["Location"].fillna("Unknown")
#df["Location"] = df["Location"]
df["Location"] = df["Location"].replace({ "NAN": "Unknown", "nan": "Unknown","Nan": "Unknown"})

In [ ]:
df["Location"].unique()[:100]

In [ ]:
df["Location"] = df["Location"].str.strip().str.lower().str.capitalize()
df["Location"]

In [ ]:
#Graphic Attacks by Year 
world_attacks_year = df["Year"].value_counts()
world_attacks_year.sort_index(ascending=True, inplace=True)
world_attacks_year.plot(color="skyblue", label ="Worldwide")

usa_attacks_year = usa_df["Year"].value_counts()
usa_attacks_year.sort_index(ascending=True, inplace=True)
usa_attacks_year.plot(color="coral", label ="United States")

x_years = df["Year"].value_counts().sort_index().index

plt.fill_between(x_years, usa_attacks_year, world_attacks_year, color="skyblue", alpha=0.25)
plt.fill_between(x_years, usa_attacks_year, color="coral", alpha=0.25)

plt.legend()
plt.tight_layout()
plt.savefig("AttacksByYear.png")